### =============================================================
### 셀 1: 필수 라이브러리 설치 및 환경 변수 설정 (RunPod Jupyter Notebook에서 가장 먼저 실행)
### =============================================================

In [ ]:
# ==============================================================================
# 0. 필수 모듈 임포트 및 환경 설정
# ==============================================================================

# RunPod 환경: runpod/pytorch:2.1.0-py3.10-cuda11.8.0-devel-ubuntu22.04

import os
import sys
import pkg_resources
import gc # 가비지 컬렉션을 위함

# pip 및 torch 업그레이드
print("환경 설정 및 필수 라이브러리 설치를 시작합니다.")
!pip install --upgrade pip torch

# ==============================================================================

# --- 1. 현재 커널의 Python 버전 확인 (참고용) ---
print(f"현재 커널의 Python 버전: {sys.version}")

# --- 2. CUDA 버전 확인 ---
print("\n--- CUDA 버전 확인 중 ---")

# --- 3. PyTorch 및 CUDA 버전 확인 (RunPod 환경에 이미 설치된 것 활용) ---
print("\n--- RunPod 환경의 PyTorch 및 CUDA 버전 확인 중 ---")
try:
    import torch # PyTorch 버전 확인을 위함
    print(f"현재 PyTorch 버전: {torch.__version__}")
    print(f"PyTorch CUDA 사용 가능 여부: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"PyTorch가 인식하는 CUDA 버전: {torch.version.cuda}")
        print(f"현재 GPU 이름: {torch.cuda.get_device_name(0)}")
    else:
        print("PyTorch가 CUDA를 인식하지 못합니다. 문제 발생 가능성이 있습니다.")
except ImportError:
    print("PyTorch가 설치되지 않았거나 임포트할 수 없습니다. 수동 설치가 필요할 수 있습니다.")

# --- 4. pip 캐시 완전 삭제 ---
print("\n--- pip 캐시 삭제 중... ---")
!pip cache purge

# --- 5. 핵심 라이브러리 최신 버전 설치 ---
!pip install -U peft trl transformers bitsandbytes typing_extensions accelerate

# --- 6. NumPy 2.0.2 버전 설치 ---
print("\n--- NumPy 2.0.2 버전 설치 중... ---")
!pip install numpy==2.0.2

# --- 7. ipykernel 설치 ---
print("\n--- ipykernel 설치 중... ---")
!pip install ipykernel

# --- 8. 나머지 필수 라이브러리 설치 (datasets, safetensors, langchain 등) ---
print("\n--- 나머지 필수 라이브러리 및 datasets, safetensors 설치 중... ---")
!pip install \
    datasets \
    safetensors \
    langchain \
    langchain-community \
    langchain-core \
    faiss-cpu

# --- 9. 가비지 컬렉션 및 GPU 캐시 정리 ---
print("\n--- 가비지 컬렉션 및 GPU 캐시 정리 중... ---")
gc.collect()
torch.cuda.empty_cache()
print("\n--- 모든 라이브러리 설치 및 환경 변수 설정 완료! ---")

# --- 10. 설치 확인 (모든 주요 라이브러리 임포트 테스트 강화) ---
print("\n--- bitsandbytes 진단 실행 중... ---")
get_ipython().system('python -m bitsandbytes')
print("\n--- 주요 라이브러리 임포트 테스트 ---")
try:
    installed_numpy_version = pkg_resources.get_distribution("numpy").version
    print(f"\n설치된 numpy 버전: {installed_numpy_version}")

    installed_typing_extensions_version = pkg_resources.get_distribution("typing_extensions").version
    print(f"설치된 typing_extensions 버전: {installed_typing_extensions_version}")

    import torch
    print(f"설치된 PyTorch 버전: {torch.__version__}")
    print(f"PyTorch CUDA 사용 가능 여부: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"PyTorch CUDA 버전: {torch.version.cuda}")

    import transformers
    print(f"transformers 버전: {transformers.__version__}")

    import bitsandbytes as bnb
    print(f"bitsandbytes 버전: {bnb.__version__}")

    import accelerate
    print(f"accelerate 버전: {accelerate.__version__}")

    import peft
    print(f"peft 버전: {peft.__version__}")

    import trl
    print(f"trl 버전: {trl.__version__}")

    import tokenizers
    print(f"tokenizers 버전: {tokenizers.__version__}")

    import safetensors
    # safetensors는 'safetensors'라는 이름으로 설치되므로, pkg_resources.get_distribution 사용
    installed_safetensors_version = pkg_resources.get_distribution("safetensors").version
    print(f"safetensors 버전: {installed_safetensors_version}")

    import datasets
    print(f"datasets 버전: {datasets.__version__}")

    import langchain
    print(f"langchain 버전: {langchain.__version__}")

    import faiss
    print(f"faiss-cpu 버전: {faiss.__version__}")

    print("\n--- 모든 주요 라이브러리 임포트 테스트 성공. ---")
except Exception as e:
    print(f"\n--- 라이브러리 확인 중 오류 발생: {e} ---")

# --- 11. 중요: Jupyter 커널 재시작 권장 ---
print("\n--- 경고: Jupyter 커널을 재시작하는 것을 강력히 권장합니다 (메뉴: Kernel -> Restart Kernel). ---")
print("재시작 후, 이 셀을 건너뛰고 바로 다음 셀(본 코드)을 실행하십시오.")

### ===================================================
### 셀 2: Vector DB 비교 및 RAG 추론 코드 (셀 1 실행 및 커널 재시작 후 실행)
### ===================================================

In [1]:
# ==============================================================================
# 1. 필수 라이브러리 임포트 및 환경 설정
# ==============================================================================

import os
import gc
import torch
import numpy as np
import time

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores.faiss import FAISS

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
# ==============================================================================
# 2. 임베딩 모델 및 벡터스토어 로드
# ==============================================================================

EMBEDDING_MODEL_NAME = "nlpai-lab/KURE-v1"
VECTORSTORE_PATH      = "./vectordb_A/"
TRAIN_INFERENCE_MODEL_NAME = "Qwen/Qwen3-8B"      
MODEL_PATH               = "./dais_model/"

# SentenceTransformer 불러오기
kure_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

class KURELangChainEmbeddings:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        # 리스트 입력 시 문서 임베딩
        return self.model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

    def embed_query(self, text):
        # 단일 쿼리 입력 시
        return self.model.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0]

    def __call__(self, inputs):
        """
        FAISS 등에서 embeddings(inputs)로 호출될 때 대응.
        - 만약 단일 문자열이면 embed_query
        - 리스트나 튜플이면 embed_documents
        """
        if isinstance(inputs, str):
            return self.embed_query(inputs)
        elif isinstance(inputs, (list, tuple)):
            return self.embed_documents(inputs)
        else:
            raise ValueError(f"Unsupported input type for embeddings: {type(inputs)}")

kure_lc_embeddings = KURELangChainEmbeddings(kure_model)

if os.path.isdir(VECTORSTORE_PATH):
    vectorstore_rag = FAISS.load_local(
        folder_path=VECTORSTORE_PATH,
        embeddings=kure_lc_embeddings,
        allow_dangerous_deserialization=True
    )
else:
    raise FileNotFoundError(f"벡터스토어 폴더가 존재하지 않습니다: {VECTORSTORE_PATH}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [3]:
# ==============================================================================
# 3. LLM 및 토크나이저 로드 (파인튜닝 때 저장된 토크나이저/모델 사용)
# ==============================================================================

# 1) 파인튜닝 저장 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    use_fast=True
)
vocab_size = len(tokenizer)

# 2) 4-bit 양자화 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    # llm_int8_enable_fp32_cpu_offload=True  # 32-bit offload 활성화
)

# 3) 베이스 모델 로드 (임베딩 크기 아직 original)
base_model = AutoModelForCausalLM.from_pretrained(
    TRAIN_INFERENCE_MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    ignore_mismatched_sizes=True
)

# 4) 베이스 모델에만 resize 호출 (PEFT 어댑터 이전)
base_model.resize_token_embeddings(vocab_size)

# 5) PEFT 어댑터 로드
inference_model = PeftModel.from_pretrained(
    base_model,
    MODEL_PATH,
    device_map="auto",
    ignore_mismatched_sizes=True
)

# 평가 모드 및 캐시 설정
inference_model.eval()
inference_model.config.use_cache = True

# 메모리/캐시 정리
gc.collect()
torch.cuda.empty_cache()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
# ==============================================================================
# 4. 평가 함수 정의
# ==============================================================================

def calculate_bleu(reference, hypothesis):
    return sentence_bleu(
        [reference.split()],
        hypothesis.split(),
        weights=(0.25, 0.25, 0.25, 0.25),
        smoothing_function=SmoothingFunction().method4
    )

def calculate_rouge(reference, hypothesis):
    return rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True).score(reference, hypothesis)['rougeL'].fmeasure

def calculate_perplexity(model, tokenizer, texts):
    total_loss, total_tokens = 0.0, 0
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=2048)
        ids = inputs["input_ids"].to(model.device)
        with torch.no_grad():
            loss = model(ids, labels=ids).loss.item()
        total_loss += loss * ids.size(1)
        total_tokens += ids.size(1)
    return float(np.exp(total_loss / total_tokens)) if total_tokens > 0 else float('inf')

In [5]:
# ==============================================================================
# 5. RunnableWithMessageHistory용 세션별 메시지 히스토리 관리
# ==============================================================================

chats_by_session_id = {}

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    hist = chats_by_session_id.get(session_id)
    if hist is None:
        hist = InMemoryChatMessageHistory()
        chats_by_session_id[session_id] = hist
    return hist

In [6]:
# ==============================================================================
# 6. RAG 추론 함수 (Top-1)
# ==============================================================================

def rag_chain_fn(inputs):
    query       = inputs["query"]
    history_msgs = inputs.get("history", [])

    # Top-1 문서 검색
    docs = vectorstore_rag.similarity_search_with_score(query, k=1)
    if docs:
        doc, score = docs[0]
        answer = doc.page_content
        top1_score = score
    else:
        answer = ""
        top1_score = 0.0

    # 대화 이력 문자열화
    history = ""
    for msg in history_msgs:
        if isinstance(msg, HumanMessage):
            history += f"[USER] {msg.content}\n"
        else:
            history += f"[DAIS] {msg.content}\n"

    # DAIS 프롬프트 조합
    prompt = f"""
[DAIS_INSTRUCTION]
너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 [OUTPUT]를 참고해서 [INPUT]에 답해줘.

[DAIS_STYLE]
1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침

[DAIS_RULE]
1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부

[DAIS_EXAMPLE]
안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!

[HISTORY]
{history}

[INPUT]
{query}

[OUTPUT]
{answer}

[CONTEXT]
"""
    tok       = tokenizer(prompt, return_tensors="pt", truncation=True)
    input_ids = tok.input_ids.to(inference_model.device)
    mask      = tok.attention_mask.to(inference_model.device)

    with torch.no_grad():
        gen = inference_model.generate(
            input_ids=input_ids,
            attention_mask=mask,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.9,
            top_p=0.3,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )
    llm_resp = tokenizer.decode(gen[0], skip_special_tokens=True)

    return {
        "llm_response": llm_resp,
        "answer":       answer,
        "top1_score":   top1_score
    }

rag_chain = RunnableLambda(rag_chain_fn)

In [7]:
# ==============================================================================
# 7. RunnableWithMessageHistory 래핑
# ==============================================================================

runnable_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_chat_history,
    input_messages_key="query",  # 사용자 질문
    history_messages_key="history"  # 메시지 히스토리 (BaseMessage 리스트)
)

In [ ]:
# ==============================================================================
# 8. 실시간 챗봇 + 평가 & 타이밍 루프
# ==============================================================================

def run_rag_with_eval_and_timing(runnable, session_id="default"):
    print("=== 실시간 RAG 챗봇 (Top-1 + 자동 평가 + 타이밍) ===")
    print("질문 입력 ('exit' 입력 시 종료 및 통계)\n")

    cos_sims, ppls, bleus, rouges, times = [], [], [], [], []

    while True:
        start = time.time()
        q = input(">> ")
        if q.strip().lower() == "exit":
            def stats(x): return (None, None, None) if not x else (max(x), min(x), np.mean(x))
            cs_max, cs_min, cs_avg = stats(cos_sims)
            ppl_max, ppl_min, ppl_avg = stats(ppls)
            b_max, b_min, b_avg = stats(bleus)
            r_max, r_min, r_avg = stats(rouges)
            t_max, t_min, t_avg = stats(times)

            print("\n=== 최종 평가 통계 ===")
            print(f"Cosine Similarity: max={cs_max:.4f}, min={cs_min:.4f}, avg={cs_avg:.4f}")
            print(f"Perplexity:        max={ppl_max:.4f}, min={ppl_min:.4f}, avg={ppl_avg:.4f}")
            print(f"BLEU-4:            max={b_max:.4f}, min={b_min:.4f}, avg={b_avg:.4f}")
            print(f"ROUGE-L:           max={r_max:.4f}, min={r_min:.4f}, avg={r_avg:.4f}")
            print(f"Response Time(s):  max={t_max:.4f}, min={t_min:.4f}, avg={t_avg:.4f}")
            break

        # 호출
        res = runnable.invoke(
            {"query": q},
            config={"configurable": {"session_id": session_id}}
        )
        resp = res["llm_response"]
        ans  = res["answer"]
        sim  = res["top1_score"]

        elapsed = time.time() - start
        times.append(elapsed)

        # 평가 지표
        cos_sims.append(sim)
        ppl = calculate_perplexity(inference_model, tokenizer, [resp]);  ppls.append(ppl)

        if ans:
            b = calculate_bleu(ans, resp)
            r = calculate_rouge(ans, resp)
        else:
            b = r = 0.0

        bleus.append(b)
        rouges.append(r)

        # 출력
        print("\n[DAIS]\n", resp)
        print("\n[문서 Top-1]\n", ans)
        print(f"\nCosine Similarity: {sim:.4f}")
        print(f"Perplexity:          {ppl:.4f}")
        print(f"BLEU-4:              {b:.4f}")
        print(f"ROUGE-L:             {r:.4f}")
        print(f"Response Time(s):    {elapsed:.4f}")
        print("-" * 60)

In [9]:
# ==============================================================================
# 9. Jupyter에서 실행
# ==============================================================================

run_rag_with_eval_and_timing(runnable_with_history, session_id="default")

=== 실시간 RAG 챗봇 (Top-1 + 자동 평가 + 타이밍) ===
질문 입력 ('exit' 입력 시 종료 및 통계)



>>  멘델의 유전법칙에 대해서 설명해줘


Error in RootListenersTracer.on_chain_end callback: KeyError('output')



[DAIS]
 

너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 를 참고해서 에 답해줘.


1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침


1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부


안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!





멘델의 유전법칙에 대해서 설명해줘


1864년 1월 6일, 한 수도사가 세상을 떠났습니다. 그는 롭 영생을 헌신하며 살았던 수도원장 그래고르 맨델이었습니다. 사람들은 그를 숭고한 인격자이자 따뜻한 마음의 친구, 모범적인 성직자로 기억했지만, 그의 진정한 업적은 잘 알려져 있지 않았습니다. 그는 어떻게 유전학의 아버지가 되었을까요? 맨델은 1822년 오스트리아의 작은 마을에서 가난한 농부의 아들로 태어났습니다. 12살에 학교에 입학했지만, 어려운 집안 형편 때문에 학비가 항상 부족했습니다. 학비를 벌기 위해 과일을 팔며 힘든 시간을 보냈습니다. 16살 때 아버지가 사고를 당해 집안이 어려워지자, 그는 병을 앓게 되었고 부모님의 집으로 돌아가 한동안 쉬어야 했습니다. 다행히 누나와 여동생의 도움으로 회복하고 학교를 졸업할 수 있었습니다. 학비와 생계에 대한 걱정을 떨쳐버리고 싶었던 그는 브루너의 성 도마 아우구스티누스 수도원으로 향했습니다. 21살의 청년 맨델은 수도원의 수련 수사가 되었고, 새로운 이름을 받았습니다. 수도원은 고립된 곳이 아니었습니다. 다양한 스승들이 있었고, 수

>>  다윈의 진화론에 대해서 알려줄래?


...


[DAIS]
 

너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 를 참고해서 에 답해줘.


1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침


1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부


안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!





양자역학의 기본 원리가 어떤거야?


양자역학은 현대 물리학의 중요한 분야로, 물질의 기본적인 성질과 그 상호작용을 다룹니다. 이 학문은 20세기 초에 등장했으며, 그 기초는 고전 물리학의 한계를 극복하려는 노력에서 시작되었습니다. 양자역학의 발전은 물질의 본질에 대한 인간의 이해를 크게 변화시켰습니다. 고대 그리스의 철학자들은 물질의 본질에 대해 다양한 이론을 제시했습니다. 예를 들어, 기원전 600년경 탈레스는 모든 것이 물로 이루어져 있다고 주장했습니다. 이후 아리스토텔레스는 물질을 구성하는 기본 요소로 '물', '불', '공기', '흙'의 네 가지 원소를 제안했습니다. 반면, 데모크리토스는 원자론을 주장하며 물질이 더 이상 나눌 수 없는 기본 단위인 원자로 구성되어 있다고 보았습니다. 이러한 초기 이론들은 후에 물리학과 화학의 발전에 큰 영향을 미쳤습니다. 18세기에는 화학의 발전이 이루어졌습니다. 앙투안 로랑 드 라부아지에는 물질의 연소 과정에서 질량 보존의 법칙을 발견하였고, 조제프 루이 프루스트는 일정 성분비의 법칙을 제안했습니다. 이러한 법칙들은 물질의 기본 단위에

>>  우주 팽창론과 블랙홀에 대해 가르쳐줘



[DAIS]
 

너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 를 참고해서 에 답해줘.


1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침


1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부


안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!





우주 팽창론과 블랙홀에 대해 가르쳐줘


우주에 대한 이해는 과학의 발전과 함께 깊어져 왔습니다. 초기에는 아인슈타인이 블랙홀이 존재하지 않을 것이라고 주장했지만, 이후의 연구들은 그 주장을 뒤집었습니다. 아인슈타인은 일반 상대성 이론을 통해 우주가 팽창하거나 수축할 수 있다고 생각했지만, 우주가 실제로 팽창하고 있다는 사실은 1929년에 에드윈 허블에 의해 관측되었습니다. 허블은 멀리 있는 은하들이 점점 더 멀어지고 있다는 것을 발견했죠. 이로 인해 아인슈타인은 자신의 우주 상수를 포기하게 되었습니다. 블랙홀의 개념은 슈바르츠 실트의 연구에서 비롯되었습니다. 그는 1차 대전 중에 블랙홀의 수학적 해를 찾아냈고, 이는 중력이 매우 강한 구역에서 빛조차 탈출할 수 없는 상황을 설명했습니다. 블랙홀은 특정 질량이 일정 반지름 이하로 수축할 때 형성됩니다. 예를 들어, 태양이 약 3킬로미터로 수축하면 블랙홀이 됩니다. 아인슈타인은 이러한 블랙홀이 존재하지 않을 것이라고 생각했지만, 후속 연구들은 질량이 큰 별들이 죽으면서 블랙홀로 변할 수 있다는 것을 보여주었습니다. 블랙홀은 단순히 

>>  exit



=== 최종 평가 통계 ===
코사인 유사도:    max=0.7482, min=0.6469, avg=0.6880
Perplexity:       max=2.9951, min=1.8388, avg=2.2197
BLEU-4:           max=0.5806, min=0.4853, avg=0.5342
ROUGE-L:          max=0.6038, min=0.2222, avg=0.3829
Response Time(s): max=169.7518, min=126.9657, avg=139.0559


In [ ]:
# 1. 멘델의 유전법칙에 대해서 설명해줘
# 2. 다윈의 진화론에 대해서 알려줄래?
# 3. 상대성 이론에 대해 알고 싶어
# 4. 양자역학의 기본 원리가 어떤거야?
# 5. 우주 팽창론과 블랙홀에 대해 가르쳐줘

# ====================================================

In [ ]:
# ==============================================================================
# 2. 임베딩 모델 및 벡터스토어 로드
# ==============================================================================

EMBEDDING_MODEL_NAME = "nlpai-lab/KURE-v1"
VECTORSTORE_PATH      = "./vectordb_B/"
TRAIN_INFERENCE_MODEL_NAME = "Qwen/Qwen3-8B"      
MODEL_PATH               = "./dais_model/"

# SentenceTransformer 불러오기
kure_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

class KURELangChainEmbeddings:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        # 리스트 입력 시 문서 임베딩
        return self.model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

    def embed_query(self, text):
        # 단일 쿼리 입력 시
        return self.model.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0]

    def __call__(self, inputs):
        """
        FAISS 등에서 embeddings(inputs)로 호출될 때 대응.
        - 만약 단일 문자열이면 embed_query
        - 리스트나 튜플이면 embed_documents
        """
        if isinstance(inputs, str):
            return self.embed_query(inputs)
        elif isinstance(inputs, (list, tuple)):
            return self.embed_documents(inputs)
        else:
            raise ValueError(f"Unsupported input type for embeddings: {type(inputs)}")

kure_lc_embeddings = KURELangChainEmbeddings(kure_model)

if os.path.isdir(VECTORSTORE_PATH):
    vectorstore_rag = FAISS.load_local(
        folder_path=VECTORSTORE_PATH,
        embeddings=kure_lc_embeddings,
        allow_dangerous_deserialization=True
    )
else:
    raise FileNotFoundError(f"벡터스토어 폴더가 존재하지 않습니다: {VECTORSTORE_PATH}")

In [23]:
# ==============================================================================
# 5. RunnableWithMessageHistory용 세션별 메시지 히스토리 관리
# ==============================================================================

chats_by_session_id = {}

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    hist = chats_by_session_id.get(session_id)
    if hist is None:
        hist = InMemoryChatMessageHistory()
        chats_by_session_id[session_id] = hist
    return hist

In [24]:
# ==============================================================================
# 7. RunnableWithMessageHistory 래핑
# ==============================================================================

runnable_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_chat_history,
    input_messages_key="query",  # 사용자 질문
    history_messages_key="history"  # 메시지 히스토리 (BaseMessage 리스트)
)

In [26]:
# ==============================================================================
# 9. Jupyter에서 실행
# ==============================================================================

run_rag_with_eval_and_timing(runnable_with_history, session_id="default")

=== 실시간 RAG 챗봇 (Top-1 + 자동 평가 + 타이밍) ===
질문 입력 ('exit' 입력 시 종료 및 통계)



>>  멘델의 유전법칙에 대해서 설명해줘



[DAIS]
 

너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 를 참고해서 에 답해줘.


1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침


1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부


안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!





멘델의 유전법칙에 대해서 설명해줘


1864년 1월 6일, 한 수도사가 세상을 떠났습니다. 그는 롭 영생을 헌신하며 살았던 수도원장 그래고르 맨델이었습니다. 사람들은 그를 숭고한 인격자이자 따뜻한 마음의 친구, 모범적인 성직자로 기억했지만, 그의 진정한 업적은 잘 알려져 있지 않았습니다. 그는 어떻게 유전학의 아버지가 되었을까요? 맨델은 1822년 오스트리아의 작은 마을에서 가난한 농부의 아들로 태어났습니다. 12살에 학교에 입학했지만, 어려운 집안 형편 때문에 학비가 항상 부족했습니다. 학비를 벌기 위해 과일을 팔며 힘든 시간을 보냈습니다. 16살 때 아버지가 사고를 당해 집안이 어려워지자, 그는 병을 앓게 되었고 부모님의 집으로 돌아가 한동안 쉬어야 했습니다. 다행히 누나와 여동생의 도움으로 회복하고 학교를 졸업할 수 있었습니다. 학비와 생계에 대한 걱정을 떨쳐버리고 싶었던 그는 브루너의 성 도마 아우구스티누스 수도원으로 향했습니다. 21살의 청년 맨델은 수도원의 수련 수사가 되었고, 새로운 이름을 받았습니다. 수도원은 고립된 곳이 아니었습니다. 다양한 스승들이 있었고, 수

>>  다윈의 진화론에 대해서 알려줄래?



[DAIS]
 

너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 를 참고해서 에 답해줘.


1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침


1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부


안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!





다윈의 진화론에 대해서 알려줄래?


1859년, 영국의 한 노신사가 출간한 책이 있습니다. 바로 찰스 다윈의 '종의 기원'인데요, 이 책은 비둘기를 개량하고 여섯 가지 종류를 만드는 이야기로 시작됩니다. 이 책은 하루 만에 매진될 정도로 큰 인기를 끌었고, 다윈은 이를 통해 인간이 비둘기를 원하는 형태로 변화시키는 일상적인 예를 들며 놀라운 과학 법칙을 설명했습니다. 이 법칙은 바로 '진화'입니다. 다윈은 '종의 기원'을 통해 인간이 다양한 생물 종의 변화를 통해 자연의 선택을 거쳐 지금의 다양성을 갖게 되었다고 주장했습니다. 자연 선택이란, 생물 종 내에서 다양한 형질이 나타나고, 그 중 환경에 적합한 형질을 가진 개체가 살아남아 번식하고, 그렇지 못한 개체는 도태되는 과정을 의미합니다. 이렇게 좋은 형질이 천천히 변화하면서 생명체가 발전해 나간다는 것이죠. 다윈은 생명이 원시적인 생명체에서 출발하여 수많은 세월 동안 자연 선택을 통해 현재의 다양성을 갖게 되었다고 말했습니다. 그리고 인간 역시 대자연에서 살아남아 번식에 성공한 평범한 생물 종에 불과하다는 점을 강조했습니다.

...


[DAIS]
 

너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 를 참고해서 에 답해줘.


1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침


1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부


안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!





양자역학의 기본 원리가 어떤거야?


양자역학은 현대 물리학의 중요한 분야로, 물질의 기본적인 성질과 그 상호작용을 다룹니다. 이 학문은 20세기 초에 등장했으며, 그 기초는 고전 물리학의 한계를 극복하려는 노력에서 시작되었습니다. 양자역학의 발전은 물질의 본질에 대한 인간의 이해를 크게 변화시켰습니다. 고대 그리스의 철학자들은 물질의 본질에 대해 다양한 이론을 제시했습니다. 예를 들어, 기원전 600년경 탈레스는 모든 것이 물로 이루어져 있다고 주장했습니다. 이후 아리스토텔레스는 물질을 구성하는 기본 요소로 '물', '불', '공기', '흙'의 네 가지 원소를 제안했습니다. 반면, 데모크리토스는 원자론을 주장하며 물질이 더 이상 나눌 수 없는 기본 단위인 원자로 구성되어 있다고 보았습니다. 이러한 초기 이론들은 후에 물리학과 화학의 발전에 큰 영향을 미쳤습니다. 18세기에는 화학의 발전이 이루어졌습니다. 앙투안 로랑 드 라부아지에는 물질의 연소 과정에서 질량 보존의 법칙을 발견하였고, 조제프 루이 프루스트는 일정 성분비의 법칙을 제안했습니다. 이러한 법칙들은 물질의 기본 단위에

>>  우주 팽창론과 블랙홀에 대해 가르쳐줘



[DAIS]
 

너의 이름은 DAIS이고, 너의 역할은 과학 AI 인플루언서야.
아래 를 참고해서 에 답해줘.


1. 복잡한 개념을 쉽게 설명
2. 팬과의 소통을 최우선
3. 쾌활하고 위트있고 유머러스
4. 가끔 반대로 질문을 던지거나 장난침


1. 반드시 한국어로, 자연스럽게 답변
2. 팬의 의도를 먼저 파악
3. Step-back & CoT 적용
4. 자신이 AI임을 밝히지 않음
5. 중복 표현 없이 핵심만 쉽게 설명
6. 의도가 모호하면 솔직하게 답변
7. 필요시 팬에게 추가 정보 요청
8. RAG 기반 답변, 신뢰 가능한 출처/링크 제공
9. 법적·도덕적·윤리적 문제 답변 거부


안녕, 난 과학 인플루언서 DAIS라고 해!
내 팬이 된 것을 후회하지 않도록, 너의 과학에 대한 모든 궁금증을 내가 다 해결해줄게!
어떤게 궁금해서 찾아왔어? 뭐든지 다 말만 해봐!





우주 팽창론과 블랙홀에 대해 가르쳐줘


우주에 대한 이해는 과학의 발전과 함께 깊어져 왔습니다. 초기에는 아인슈타인이 블랙홀이 존재하지 않을 것이라고 주장했지만, 이후의 연구들은 그 주장을 뒤집었습니다. 아인슈타인은 일반 상대성 이론을 통해 우주가 팽창하거나 수축할 수 있다고 생각했지만, 우주가 실제로 팽창하고 있다는 사실은 1929년에 에드윈 허블에 의해 관측되었습니다. 허블은 멀리 있는 은하들이 점점 더 멀어지고 있다는 것을 발견했죠. 이로 인해 아인슈타인은 자신의 우주 상수를 포기하게 되었습니다. 블랙홀의 개념은 슈바르츠 실트의 연구에서 비롯되었습니다. 그는 1차 대전 중에 블랙홀의 수학적 해를 찾아냈고, 이는 중력이 매우 강한 구역에서 빛조차 탈출할 수 없는 상황을 설명했습니다. 블랙홀은 특정 질량이 일정 반지름 이하로 수축할 때 형성됩니다. 예를 들어, 태양이 약 3킬로미터로 수축하면 블랙홀이 됩니다. 아인슈타인은 이러한 블랙홀이 존재하지 않을 것이라고 생각했지만, 후속 연구들은 질량이 큰 별들이 죽으면서 블랙홀로 변할 수 있다는 것을 보여주었습니다. 블랙홀은 단순히 

>>  exit



=== 최종 평가 통계 ===
코사인 유사도:    max=0.7482, min=0.6469, avg=0.6880
Perplexity:       max=2.9449, min=1.8079, avg=2.2086
BLEU-4:           max=0.5867, min=0.4878, avg=0.5374
ROUGE-L:          max=0.6038, min=0.2222, avg=0.3824
Response Time(s): max=133.6208, min=126.6415, avg=129.7084


In [ ]:
# 1. 멘델의 유전법칙에 대해서 설명해줘
# 2. 다윈의 진화론에 대해서 알려줄래?
# 3. 상대성 이론에 대해 알고 싶어
# 4. 양자역학의 기본 원리가 어떤거야?
# 5. 우주 팽창론과 블랙홀에 대해 가르쳐줘